# fMRI1 -- Compute the baseline decodability (within-subject!) of input stimuli features (COLOR, ORIENTATION, PITCH, CONSTANT)

## Use classifications
## Using Ciric-style postprocessing

## Takuya Ito
#### 01/17/2019

In [129]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import multiprocessing as mp
import scipy.stats as stats
import nibabel as nib
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.api as sm
import sklearn.svm as svm
import statsmodels.sandbox.stats.multicomp as mc
import sklearn
from sklearn.feature_selection import f_classif
import seaborn as sns
import h5py
os.sys.path.append('glmScripts/')
import taskGLMPipeline as tgp
os.sys.path.append('utils/')
import loadExperimentalData as led
sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


In [130]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']

## TEMPORARY - WAITING FOR REST OF CODE TO FINISH RUNNING
subjNums = subjNums[:83]



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 0.0 Define functions for loading data

In [131]:
def loadTrialData(subj):
#     x = tgp.loadTaskTiming(subj,'betaSeries')
#     stimIndex = np.asarray(x['stimIndex'])
#     ind = np.where(stimIndex=='motorResponse')[0]
    
    datadir = basedir + 'data/postProcessing/hcpPostProcCiric/'
    h5f = h5py.File(datadir + subj + '_glmOutput_data.h5','r')
    data = h5f['taskRegression/betaSeries_24pXaCompCorXVolterra_taskReg_betas_canonical'][:].copy()
    # Probe activations are starting from index 128 (first 128 are encoding activations)
    data = data[:,128:].copy()
#     data = np.loadtxt(datadir + subj + '_motorResponse_taskBetas_Surface64k_GSR.csv',delimiter=',')
#     data = data[:,-4:]
    h5f.close()
    return data
        

## 0.1 Load data

In [132]:
# gsr = True
nTrials = 384
data_task = np.zeros((len(glasser2),nTrials,len(subjNums)))

scount = 0
for subj in subjNums:
    if scount%4==0: print 'Loading subject', scount, '/', len(subjNums)
    data_task[:,:,scount] = loadTrialData(subj)
    scount += 1

Loading subject 0 / 83
Loading subject 4 / 83
Loading subject 8 / 83
Loading subject 12 / 83
Loading subject 16 / 83
Loading subject 20 / 83
Loading subject 24 / 83
Loading subject 28 / 83
Loading subject 32 / 83
Loading subject 36 / 83
Loading subject 40 / 83
Loading subject 44 / 83
Loading subject 48 / 83
Loading subject 52 / 83
Loading subject 56 / 83
Loading subject 60 / 83
Loading subject 64 / 83
Loading subject 68 / 83
Loading subject 72 / 83
Loading subject 76 / 83
Loading subject 80 / 83


# 1.0 Define functions for motor response decodings

In [153]:
from scipy.spatial.distance import cdist

def inputStimuliDecodings((data, subj, stimkey, ncvs)):
    """
    Run a within-subject classification
    Assumes data is a space X feature matrix
    Decode responses on each hand separately from CPRO data
    """
    
    df_task = led.loadExperimentalData(subj) 
    stim1_values = df_task['Stim1_' + stimkey].values
    stim2_values = df_task['Stim2_' + stimkey].values
    stim_ind = {}
    labels = []
    stimcount = 0
    for stim1 in np.unique(stim1_values):
        for stim2 in np.unique(stim2_values):
            stim_combo = stim1 + stim2
            # Identify trials in which stim1 and stim2 were presented
            stim_ind[stim_combo] = np.intersect1d(np.where(stim1_values==stim1)[0],np.where(stim2_values==stim2)[0])
            labels.extend(np.repeat(stimcount,len(stim_ind[stim_combo])))
            stimcount += 1
    
    svm_mat = []
    for stim_combo in stim_ind:
        svm_mat.extend(data[:,stim_ind[stim_combo]].T)
    svm_mat = np.asarray(svm_mat)
        
    # Spatially demean matrix across features
    samplemean = np.mean(svm_mat,axis=1)
    samplemean.shape = (len(samplemean),1)
    svm_mat = svm_mat - samplemean

    scores = randomSplitLOOBaselineCV(ncvs, svm_mat, labels)

    return scores

def randomSplitLOOBaselineCV(ncvs, svm_mat, labels):
    """
    Runs cross validation for a within-subject SVM analysis
    Using boot-strapped CV
    Approx. 80% train set, 20% test set
    """
    
    # Data set might be unbalanced, so find minimium number of unique samples
    maxpossible = len(labels)
    for i in np.unique(labels):
        if np.sum(labels==i)<maxpossible:
            maxpossible = np.sum(labels==i)
    min_unique_samples = maxpossible
    # Train set is approximately 80%
    n_trainset_per_cond = np.floor(min_unique_samples*.8)
    # Test set is the remaining samples
    n_testset_per_cond = min_unique_samples - n_trainset_per_cond
    
    accuracies = []
    for cv in range(ncvs):
        # Define training and test set labels
        train_ind = []
        trainlabels = []
        for i in np.unique(labels):
            ind = np.where(labels==i)[0]
            train_ind.extend(np.random.choice(ind,int(n_trainset_per_cond),replace=False))
            trainlabels.extend(np.repeat(i,n_trainset_per_cond))
        train_ind = np.asarray(train_ind)
        test_ind = np.delete(np.arange(len(labels)),train_ind)
        testlabels = np.delete(labels,train_ind)
        
        # Define train set and test set matrices
        trainset = svm_mat[train_ind,:]
        testset = svm_mat[test_ind,:]
        
        # Normalize trainset and testset using trainset stats
        mean = np.mean(svm_mat[train_ind,:],axis=0)
        mean.shape = (1,len(mean))
        std = np.std(svm_mat[train_ind,:],axis=0)
        std.shape = (1,len(std))

        trainset = np.divide((trainset - mean),std)
        testset = np.divide((testset - mean),std)

#         ## Feature selection and downsampling
#         unique_labels = np.unique(labels)
#         feat1_labs = np.where(trainlabels==unique_labels[0])[0]
#         feat2_labs = np.where(trainlabels==unique_labels[1])[0]
#         # Perform t-test
#         t, p = stats.ttest_rel(trainset[feat1_labs,:],trainset[feat2_labs,:],axis=0)
#         h0, qs = mc.fdrcorrection0(p)
# #         h0 = p<0.1
# #         # Construct feature masks
# #         feat1_mask = np.multiply(t>0,h0).astype(bool)
# #         feat2_mask = np.multiply(t<0,h0).astype(bool)
#         feat1_mask = t>0
#         feat2_mask = t<0
#         # Downsample training set into original vertices into 2 ROI signals
#         trainset_downsampled = np.zeros((trainset.shape[0],2))
#         trainset_downsampled[:,0] = np.nanmean(trainset[:,feat1_mask],axis=1)
#         trainset_downsampled[:,1] = np.nanmean(trainset[:,feat2_mask],axis=1)
#         trainset_downsampled = trainset[:,h0]
#         # Downsample test set into original vertices
#         testset_downsampled = np.zeros((testset.shape[0],2))
#         testset_downsampled[:,0] = np.nanmean(testset[:,feat1_mask],axis=1)
#         testset_downsampled[:,1] = np.nanmean(testset[:,feat2_mask],axis=1)
#         testset_downsampled = testset[:,h0]

#         if np.sum(feat1_mask)==0 or np.sum(feat2_mask==0):
#             accuracies.append(_decoding((trainset,testset,trainlabels,testlabels)))
#         else:
#             accuracies.append(_decoding((trainset_downsampled,testset_downsampled,trainlabels,testlabels)))
        
        accuracies.append(_decoding((trainset,testset,trainlabels,testlabels)))
        
    return np.mean(accuracies)

def _decoding((trainset,testset,trainlabels,testlabels)):

# #     clf = sklearn.linear_model.LogisticRegression()
#     clf = svm.SVC(C=1.0, kernel='linear')

#     clf.fit(trainset,trainlabels)
#     predictions = clf.predict(testset)
#     acc = predictions==testlabels
#     acc = np.mean(acc)

    unique_cond = np.unique(trainlabels)
    rdm = np.zeros((len(unique_cond),len(unique_cond)))
    acc = []
    for cond1 in unique_cond:
        mismatches = []
        prototype_ind = np.where(trainlabels==cond1)[0]
        prototype = np.mean(trainset[prototype_ind,:],axis=0)
        for cond2 in unique_cond:
            test_ind = np.where(testlabels==cond2)[0]
            test = np.mean(testset[test_ind,:],axis=0)
            if cond1 == cond2: 
                correct = stats.spearmanr(prototype,test)[0]
            else:
                mismatches.append(stats.spearmanr(prototype,test)[0])
        
        if correct > np.max(mismatches): 
            acc.append(1.0)
        else:
            acc.append(0.0)
    
#     # Mahalanobis distance
#     unique_cond = np.unique(trainlabels)
#     rdm = np.zeros((len(unique_cond),len(unique_cond)))
#     acc = []
#     for cond1 in unique_cond:
#         mismatches = []
#         prototype_ind = np.where(trainlabels==cond1)[0]
#         prototype = trainset[prototype_ind,:]
#         for cond2 in unique_cond:
#             test_ind = np.where(testlabels==cond2)[0]
#             test = testset[test_ind,:]
#             if cond1 == cond2: 
# #                 correct = np.linalg.norm(prototype-test)
#                 correct = np.mean(cdist(prototype,test,'mahalanobis',VI=np.linalg.pinv(np.cov(np.vstack((prototype,test))))))
#             else:
# #                 mismatches.append(np.linalg.norm(prototype-test))
#                 mismatches.append(np.mean(cdist(prototype,test,'mahalanobis',VI=np.linalg.pinv(np.cov(np.vstack((prototype,test)))))))
        
#         if correct > np.max(mismatches): 
#             acc.append(1.0)
#         else:
#             acc.append(0.0)
    
    return acc

## 1.1 Run across subject decoding on color input stimuli 

In [154]:
nproc = 10
rois = []
rois.extend(np.where(networkdef==networkmappings['vis1'])[0])
rois.extend(np.where(networkdef==networkmappings['vis2'])[0])
statistics_color = np.zeros((len(rois),len(subjNums)))
stim = 'Color'
nCVs = 10 # These are bootstrapped CVs

roicount = 0
for roi in rois:
    print 'Running within-subject decoding of', stim, 'inputs on ROI', roi
    roi_ind = np.where(glasser2==roi+1)[0]
    scount = 0
    inputs = []
    for subj in subjNums:
        roi_data = data_task[roi_ind,:,scount]
        inputs.append((roi_data,subj,stim,nCVs))
        scount += 1
    
    # Run in parallel
    pool = mp.Pool(processes=nproc)
    results = pool.map_async(inputStimuliDecodings,inputs).get()
    pool.close()
    pool.join()
    
    # Store in array
    statistics_color[roicount,:] = np.asarray(results).copy()
    
    roicount += 1

Running within-subject decoding of Color inputs on ROI 0
Running within-subject decoding of Color inputs on ROI 120
Running within-subject decoding of Color inputs on ROI 141
Running within-subject decoding of Color inputs on ROI 180
Running within-subject decoding of Color inputs on ROI 300
Running within-subject decoding of Color inputs on ROI 321
Running within-subject decoding of Color inputs on ROI 1
Running within-subject decoding of Color inputs on ROI 2
Running within-subject decoding of Color inputs on ROI 3
Running within-subject decoding of Color inputs on ROI 4
Running within-subject decoding of Color inputs on ROI 5
Running within-subject decoding of Color inputs on ROI 6
Running within-subject decoding of Color inputs on ROI 12
Running within-subject decoding of Color inputs on ROI 15
Running within-subject decoding of Color inputs on ROI 16
Running within-subject decoding of Color inputs on ROI 17
Running within-subject decoding of Color inputs on ROI 18
Running within-s

# Compute statistics

In [155]:
ts, ps = stats.ttest_1samp(statistics_color,0.25,axis=1)

for i in range(len(rois)):
    if ts[i] > 0:
        ps[i] = ps[i]/2.0
    else:
        ps[i] = 1.0 - ps[i]/2.0
        
qs = mc.fdrcorrection0(ps)[1]

for i in range(len(rois)):
    if qs[i] < 0.05:
        print 'ROI', rois[i], 'Accuracy:', np.mean(statistics_color[i,:]), '| p =', qs[i]
    

ROI 16 Accuracy: 0.286746987952 | p = 0.0474430535698
ROI 47 Accuracy: 0.296385542169 | p = 0.0392707806335
ROI 162 Accuracy: 0.293674698795 | p = 0.0392707806335
ROI 183 Accuracy: 0.295180722892 | p = 0.0420285007347
ROI 195 Accuracy: 0.292771084337 | p = 0.0293376879543
ROI 335 Accuracy: 0.291265060241 | p = 0.0392707806335
ROI 342 Accuracy: 0.286445783133 | p = 0.0474430535698


## 1.1 Run across subject decoding on ORIENTATION input stimuli 

In [156]:
nproc = 10
rois = []
rois.extend(np.where(networkdef==networkmappings['vis1'])[0])
rois.extend(np.where(networkdef==networkmappings['vis2'])[0])
statistics_ori = np.zeros((len(rois),len(subjNums)))
stim = 'Ori'
nCVs = 10 # These are bootstrapped CVs

roicount = 0
for roi in rois:
    print 'Running within-subject decoding of', stim, 'inputs on ROI', roi
    roi_ind = np.where(glasser2==roi+1)[0]
    scount = 0
    inputs = []
    for subj in subjNums:
        roi_data = data_task[roi_ind,:,scount]
        inputs.append((roi_data,subj,stim,nCVs))
        scount += 1
    
    # Run in parallel
    pool = mp.Pool(processes=nproc)
    results = pool.map_async(inputStimuliDecodings,inputs).get()
    pool.close()
    pool.join()
    
    # Store in array
    statistics_ori[roicount,:] = np.asarray(results).copy()
    
    roicount += 1

Running within-subject decoding of Ori inputs on ROI 0
Running within-subject decoding of Ori inputs on ROI 120
Running within-subject decoding of Ori inputs on ROI 141
Running within-subject decoding of Ori inputs on ROI 180
Running within-subject decoding of Ori inputs on ROI 300
Running within-subject decoding of Ori inputs on ROI 321
Running within-subject decoding of Ori inputs on ROI 1
Running within-subject decoding of Ori inputs on ROI 2
Running within-subject decoding of Ori inputs on ROI 3
Running within-subject decoding of Ori inputs on ROI 4
Running within-subject decoding of Ori inputs on ROI 5
Running within-subject decoding of Ori inputs on ROI 6
Running within-subject decoding of Ori inputs on ROI 12
Running within-subject decoding of Ori inputs on ROI 15
Running within-subject decoding of Ori inputs on ROI 16
Running within-subject decoding of Ori inputs on ROI 17
Running within-subject decoding of Ori inputs on ROI 18
Running within-subject decoding of Ori inputs on R

# Compute statistics

In [157]:
ts, ps = stats.ttest_1samp(statistics_ori,0.25,axis=1)

for i in range(len(rois)):
    if ts[i] > 0:
        ps[i] = ps[i]/2.0
    else:
        ps[i] = 1.0 - ps[i]/2.0
        
qs = mc.fdrcorrection0(ps)[1]

for i in range(len(rois)):
    if qs[i] < 0.05:
        print 'ROI', rois[i], 'Accuracy:', np.mean(statistics_ori[i,:]), '| p =', qs[i]
    

ROI 0 Accuracy: 0.447891566265 | p = 5.98366541108e-17
ROI 141 Accuracy: 0.279518072289 | p = 0.0257902645086
ROI 180 Accuracy: 0.432530120482 | p = 5.0661869133e-16
ROI 3 Accuracy: 0.500301204819 | p = 7.45637552833e-25
ROI 4 Accuracy: 0.474096385542 | p = 4.59075972478e-20
ROI 5 Accuracy: 0.465662650602 | p = 2.81215676811e-22
ROI 6 Accuracy: 0.296084337349 | p = 0.00668176455702
ROI 18 Accuracy: 0.282831325301 | p = 0.0122699384092
ROI 19 Accuracy: 0.332228915663 | p = 4.44407527731e-07
ROI 20 Accuracy: 0.329518072289 | p = 4.80335042161e-06
ROI 21 Accuracy: 0.298192771084 | p = 0.00318214559654
ROI 151 Accuracy: 0.285843373494 | p = 0.00947364323958
ROI 157 Accuracy: 0.32921686747 | p = 1.65150173016e-05
ROI 158 Accuracy: 0.278012048193 | p = 0.0456733386495
ROI 181 Accuracy: 0.294879518072 | p = 0.00780533425494
ROI 183 Accuracy: 0.491265060241 | p = 1.92410862484e-22
ROI 184 Accuracy: 0.493975903614 | p = 1.25159981758e-21
ROI 185 Accuracy: 0.467771084337 | p = 3.00461010377e-20


## 1.1 Run across subject decoding on PITCH input stimuli 

In [158]:
nproc = 10
rois = np.where(networkdef==networkmappings['aud'])[0]
statistics_pitch = np.zeros((len(rois),len(subjNums)))
stim = 'Pitch'
nCVs = 10 # These are bootstrapped CVs

roicount = 0
for roi in rois:
    print 'Running within-subject decoding of', stim, 'inputs on ROI', roi
    roi_ind = np.where(glasser2==roi+1)[0]
    scount = 0
    inputs = []
    for subj in subjNums:
        roi_data = data_task[roi_ind,:,scount]
        inputs.append((roi_data,subj,stim,nCVs))
        scount += 1
    
    # Run in parallel
    pool = mp.Pool(processes=nproc)
    results = pool.map_async(inputStimuliDecodings,inputs).get()
    pool.close()
    pool.join()
    
    # Store in array
    statistics_pitch[roicount,:] = np.asarray(results).copy()
    
    roicount += 1

Running within-subject decoding of Pitch inputs on ROI 23
Running within-subject decoding of Pitch inputs on ROI 102
Running within-subject decoding of Pitch inputs on ROI 103
Running within-subject decoding of Pitch inputs on ROI 106
Running within-subject decoding of Pitch inputs on ROI 123
Running within-subject decoding of Pitch inputs on ROI 172
Running within-subject decoding of Pitch inputs on ROI 173
Running within-subject decoding of Pitch inputs on ROI 174
Running within-subject decoding of Pitch inputs on ROI 203
Running within-subject decoding of Pitch inputs on ROI 282
Running within-subject decoding of Pitch inputs on ROI 286
Running within-subject decoding of Pitch inputs on ROI 303
Running within-subject decoding of Pitch inputs on ROI 352
Running within-subject decoding of Pitch inputs on ROI 353
Running within-subject decoding of Pitch inputs on ROI 354


# Compute statistics

In [159]:
ts, ps = stats.ttest_1samp(statistics_pitch,0.25,axis=1)

for i in range(len(rois)):
    if ts[i] > 0:
        ps[i] = ps[i]/2.0
    else:
        ps[i] = 1.0 - ps[i]/2.0
        
qs = mc.fdrcorrection0(ps)[1]

for i in range(len(rois)):
    if qs[i] < 0.05:
        print 'ROI', rois[i], 'Accuracy:', np.mean(statistics_pitch[i,:]), '| p =', qs[i]
    

ROI 23 Accuracy: 0.490662650602 | p = 6.38094684199e-23
ROI 102 Accuracy: 0.335843373494 | p = 6.03508795164e-07
ROI 103 Accuracy: 0.407530120482 | p = 4.47172242895e-13
ROI 106 Accuracy: 0.363253012048 | p = 5.09042487137e-09
ROI 123 Accuracy: 0.556024096386 | p = 3.73323217061e-43
ROI 172 Accuracy: 0.519578313253 | p = 7.47886447453e-29
ROI 173 Accuracy: 0.550903614458 | p = 1.11218233976e-41
ROI 174 Accuracy: 0.536144578313 | p = 1.23318943555e-28
ROI 203 Accuracy: 0.458734939759 | p = 1.33295693468e-23
ROI 282 Accuracy: 0.309939759036 | p = 9.43365908384e-05
ROI 286 Accuracy: 0.418373493976 | p = 1.95190473318e-17
ROI 303 Accuracy: 0.538554216867 | p = 5.48198116446e-34
ROI 352 Accuracy: 0.474096385542 | p = 3.48174608584e-21
ROI 353 Accuracy: 0.521385542169 | p = 2.96493335498e-32
ROI 354 Accuracy: 0.570481927711 | p = 7.43584508404e-37


## 1.1 Run across subject decoding on CONSTANT input stimuli 

In [160]:
nproc = 10
rois = np.where(networkdef==networkmappings['aud'])[0]
statistics_constant = np.zeros((len(rois),len(subjNums)))
stim = 'Constant'
nCVs = 10 # These are bootstrapped CVs

roicount = 0
for roi in rois:
    print 'Running within-subject decoding of', stim, 'inputs on ROI', roi
    roi_ind = np.where(glasser2==roi+1)[0]
    scount = 0
    inputs = []
    for subj in subjNums:
        roi_data = data_task[roi_ind,:,scount]
        inputs.append((roi_data,subj,stim,nCVs))
        scount += 1
    
    # Run in parallel
    pool = mp.Pool(processes=nproc)
    results = pool.map_async(inputStimuliDecodings,inputs).get()
    pool.close()
    pool.join()
    
    # Store in array
    statistics_constant[roicount,:] = np.asarray(results).copy()
    
    roicount += 1

Running within-subject decoding of Constant inputs on ROI 23
Running within-subject decoding of Constant inputs on ROI 102
Running within-subject decoding of Constant inputs on ROI 103
Running within-subject decoding of Constant inputs on ROI 106
Running within-subject decoding of Constant inputs on ROI 123
Running within-subject decoding of Constant inputs on ROI 172
Running within-subject decoding of Constant inputs on ROI 173
Running within-subject decoding of Constant inputs on ROI 174
Running within-subject decoding of Constant inputs on ROI 203
Running within-subject decoding of Constant inputs on ROI 282
Running within-subject decoding of Constant inputs on ROI 286
Running within-subject decoding of Constant inputs on ROI 303
Running within-subject decoding of Constant inputs on ROI 352
Running within-subject decoding of Constant inputs on ROI 353
Running within-subject decoding of Constant inputs on ROI 354


# Compute statistics

In [161]:
ts, ps = stats.ttest_1samp(statistics_constant,0.25,axis=1)

for i in range(len(rois)):
    if ts[i] > 0:
        ps[i] = ps[i]/2.0
    else:
        ps[i] = 1.0 - ps[i]/2.0
        
qs = mc.fdrcorrection0(ps)[1]

for i in range(len(rois)):
    if qs[i] < 0.05:
        print 'ROI', rois[i], 'Accuracy:', np.mean(statistics_constant[i,:]), '| p =', qs[i]
    

ROI 23 Accuracy: 0.371385542169 | p = 6.76177099685e-12
ROI 102 Accuracy: 0.301506024096 | p = 0.000548737927107
ROI 103 Accuracy: 0.342168674699 | p = 1.12474783728e-08
ROI 106 Accuracy: 0.357831325301 | p = 2.83728244847e-09
ROI 123 Accuracy: 0.433734939759 | p = 3.75293301282e-16
ROI 172 Accuracy: 0.397891566265 | p = 2.89309621772e-14
ROI 173 Accuracy: 0.378012048193 | p = 1.20418319216e-13
ROI 174 Accuracy: 0.385240963855 | p = 2.91345772426e-11
ROI 203 Accuracy: 0.335240963855 | p = 7.87698904668e-07
ROI 282 Accuracy: 0.282228915663 | p = 0.0185211012877
ROI 286 Accuracy: 0.338554216867 | p = 2.32445671677e-07
ROI 303 Accuracy: 0.42078313253 | p = 2.6450131089e-16
ROI 352 Accuracy: 0.34578313253 | p = 3.90922961187e-08
ROI 353 Accuracy: 0.349698795181 | p = 2.39199189901e-09
ROI 354 Accuracy: 0.432831325301 | p = 2.51863014232e-14


## 1.3 Map accuracies back to cortical surface

In [162]:
# # Put all data into a single matrix (since we only run a single classification)
# lefthand = np.zeros((glasser2.shape[0],3))
# righthand = np.zeros((glasser2.shape[0],3))

# roicount = 0
# for roi in rois:
#     # Print significant parcel number
#     vertex_ind = np.where(glasser2==roi)[0]
#     lefthand[vertex_ind,0] = statistics_lh[roicount,0]
#     lefthand[vertex_ind,1] = statistics_lh[roicount,1]
#     lefthand[vertex_ind,2] = statistics_lh[roicount,2]

#     righthand[vertex_ind,0] = statistics_rh[roicount,0]
#     righthand[vertex_ind,1] = statistics_rh[roicount,1]
#     righthand[vertex_ind,2] = statistics_rh[roicount,2]

#     roicount += 1

    
# #### 
# # Write file to csv and run wb_command
# outdir = '/projects3/SRActFlow/data/results/WithinSubject_MotorDecoding/'
# filename = 'smnDecodingsLH_v2'
# np.savetxt(outdir + filename + '.csv', lefthand,fmt='%s')
# wb_file = filename + '.dscalar.nii'
# wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
# os.system(wb_command)

# outdir = '/projects3/SRActFlow/data/results/WithinSubject_MotorDecoding/'
# filename = 'smnDecodingsRH_v2'
# np.savetxt(outdir + filename + '.csv', righthand,fmt='%s')
# wb_file = filename + '.dscalar.nii'
# wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
# os.system(wb_command)


